In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/toxic_comms.csv')

In [ ]:
df.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [ ]:
df.shape

(14412, 2)

In [5]:
import nltk

In [4]:
from sklearn.model_selection import train_test_split

In [6]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=0)

In [21]:
sentence = df.iloc[1]["comment"]

In [22]:
sentence

'Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n'

In [23]:
from nltk.tokenize import word_tokenize

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
sentence_tokenized = word_tokenize(sentence,language='russian')

In [ ]:
sentence_tokenized

['Хохлы',
 ',',
 'это',
 'отдушина',
 'затюканого',
 'россиянина',
 ',',
 'мол',
 ',',
 'вон',
 ',',
 'а',
 'у',
 'хохлов',
 'еще',
 'хуже',
 '.',
 'Если',
 'бы',
 'хохлов',
 'не',
 'было',
 ',',
 'кисель',
 'их',
 'бы',
 'придумал',
 '.']

In [8]:
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
sentence_stopword = [word for word in sentence_tokenized if not word in stopwords.words('russian')]

In [26]:
sentence_stopword

['Хохлы',
 ',',
 'это',
 'отдушина',
 'затюканого',
 'россиянина',
 ',',
 'мол',
 ',',
 'вон',
 ',',
 'хохлов',
 'хуже',
 '.',
 'Если',
 'хохлов',
 ',',
 'кисель',
 'придумал',
 '.']

In [9]:
from nltk.stem import PorterStemmer 

In [10]:
ps = PorterStemmer()

In [27]:
sentence_lower = [word.lower() for word in sentence_stopword]

In [28]:
sentences = []
for i in sentence_lower:
  if i == ',' or i =='.':
    pass
  else:
    sentences.append(i)

In [ ]:
sentences

['хохлы',
 'это',
 'отдушина',
 'затюканого',
 'россиянина',
 'мол',
 'вон',
 'хохлов',
 'хуже',
 'если',
 'хохлов',
 'кисель',
 'придумал']

In [11]:
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language="russian")

In [29]:
sentences_stem = [snowball.stem(i) for i in sentences]

In [ ]:
sentences_stem

['хохл',
 'эт',
 'отдушин',
 'затюкан',
 'россиянин',
 'мол',
 'вон',
 'хохл',
 'хуж',
 'есл',
 'хохл',
 'кисел',
 'придума']

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
def tokenize_sentence(sentence):
  sentence_tokenized = word_tokenize(sentence,language='russian')
  sentence_stopword = [word for word in sentence_tokenized if not word in stopwords.words('russian')]
  sentence_lower = [word.lower() for word in sentence_stopword]
  sentences = []
  for i in sentence_lower:
    if i == ',' or i =='.':
      pass
    else:
      sentences.append(i)
  token = [snowball.stem(i) for i in sentences]
  return token

In [ ]:
tokenize_sentence(df.iloc[1]["comment"])

['хохл',
 'эт',
 'отдушин',
 'затюкан',
 'россиянин',
 'мол',
 'вон',
 'хохл',
 'хуж',
 'есл',
 'хохл',
 'кисел',
 'придума']

In [14]:
cv = CountVectorizer(tokenizer=lambda x: tokenize_sentence(x))

In [30]:
df_train.iloc[-1]['comment']

'Передайте своему пидарасу понасенко, что он спалился как кремлевский пидар, когда на Шульман наехал. у меня всё\n'

In [31]:
features = cv.fit_transform(df_train['comment'])

In [32]:
features.toarray()

array([[2, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [33]:
from xgboost import XGBClassifier

In [34]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=lambda x: tokenize_sentence(x))),
    ('model', XGBClassifier(random_state=0))
])

In [ ]:
model_pipeline.fit(df_train['comment'], df_train['toxic'])

Pipeline(steps=[('vectorizer',
                 CountVectorizer(tokenizer=<function <lambda> at 0x7f072bab4680>)),
                ('model', XGBClassifier())])

In [ ]:
from sklearn.metrics import precision_score, recall_score, precision_recall_curve

In [ ]:
precision_score(y_true=df_test['toxic'], y_pred=model_pipeline.predict(df_test['comment']))

0.8795620437956204

In [35]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=lambda x: tokenize_sentence(x))),
    ('model', GridSearchCV(
        estimator=XGBClassifier(random_state=0),
        param_grid= {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        },
        cv=3,
        verbose=4
        )
    )
])

In [ ]:
grid_pipeline.fit(df_train['comment'], df_train['toxic'])

Fitting 3 folds for each of 405 candidates, totalling 1215 fits
[CV 1/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.755 total time=   2.2s
[CV 2/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.763 total time=   2.2s
[CV 3/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.746 total time=   2.3s
[CV 1/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.8;, score=0.755 total time=   2.2s
[CV 2/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.8;, score=0.763 total time=   2.2s
[CV 3/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.8;, score=0.747 total time=   2.2s
[CV 1/3] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=1.0;, score=0.751 total time=   2.1s
[CV 2/3] END colsample_bytree=0.6, gamma=0.5, max_dept

Pipeline(steps=[('vectorizer',
                 CountVectorizer(tokenizer=<function <lambda> at 0x7f071fdb7050>)),
                ('model',
                 GridSearchCV(cv=3, estimator=XGBClassifier(),
                              param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                                          'gamma': [0.5, 1, 1.5, 2, 5],
                                          'max_depth': [3, 4, 5],
                                          'min_child_weight': [1, 5, 10],
                                          'subsample': [0.6, 0.8, 1.0]},
                              verbose=4))])

In [47]:
grid_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=lambda x: tokenize_sentence(x))),
    ('model', XGBClassifier(random_state=0, colsample_bytree=1.0, gamma=1.5, max_depth=5, min_child_weight=1, subsample=0.8))         
])
#colsample_bytree=1.0, gamma=1.5, max_depth=5, min_child_weight=1, subsample=0.8;, score=0.787

In [48]:
grid_pipeline.fit(df_train['comment'], df_train['toxic'])

Pipeline(steps=[('vectorizer',
                 CountVectorizer(tokenizer=<function <lambda> at 0x7fcf8f2729e0>)),
                ('model',
                 XGBClassifier(colsample_bytree=1.0, gamma=1.5, max_depth=5,
                               subsample=0.8))])

In [ ]:
precision_score(y_true=df_test['toxic'], y_pred=grid_pipeline.predict(df_test['comment']))

0.8821603927986906

In [ ]:
grid_pipeline.predict(['бля'])

array([1.])

In [38]:
import joblib

In [49]:
import dill

In [51]:
dill.dump (grid_pipeline, open('model_grid_pipeline.pkl', 'wb'))

In [53]:
with open('/content/model_grid_pipeline.pkl', 'rb') as f:
	model = dill.load(f)

In [77]:
model.predict([""])

array([0.])